<a href="https://colab.research.google.com/github/rfahrn/Shared_Task/blob/main/Shared_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd

In [89]:
train = pd.read_csv('./data/offenseval-ar-training-v1.tsv', sep='\t')

# train.head(10)
# train.iloc[2930:2940]
# len(train)

In [ ]:
# !pip install emoji
# !pip install DSAraby 
# !pip install Tashaphyne

# Preprocessing

In [126]:
import re
test = "؟؟؟اااا222"
PUNCTUATION = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''

# gsub("(.)\\1{2,}", "\\1", F)
# re.sub(r'(.)\\1{2,}', r'\\1', test)
# re.sub(r'(\D)\\1{2,}', r'\\1', test)
# re.sub(r'(.)\\1{2,}', r'\1', test)
# re.sub(r'(.)\\1+', r'\1', test)
# re.sub("(\D)\1{2,}", "\\1", test)
# re.sub(r'(.)\1+', r'\1', '12233322155552')


'؟؟؟اا222'

In [127]:
import emoji
PUNCTUATION = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''


def remove_emoji(tweet):
    return emoji.get_emoji_regexp().sub(u'', tweet)


def remove_lf(tweet):
    return tweet.replace('<LF>', ' ')


def remove_repeat_chars(tweet):
    """Removes only non-digit, non-punctuation characters that are repeated more than once, since double characters can appear in correct spelling"""
    new = ''

    for char in tweet:
        if char.isdigit():
            new += char
        elif char in PUNCTUATION:
            new += char
        else:
            if not new.endswith(char+char):
                new += char

    return new


# train['word_count'] = train['tweet'].apply(lambda x: len(str(x).split(" ")))
# train['char_count'] = train['tweet'].str.len()  # this also includes spaces
# train['avg_char_per_word'] = train['tweet'].apply(lambda x: avg_word(x))
# train['emoji_count'] = train['tweet'].apply(lambda x: emoji_counter(x))

train['tweet'] = train['tweet'].apply(remove_lf)
train['tweet'] = train['tweet'].apply(remove_repeat_chars)

# Make one df with emoji and one without to compare performance
train_no_emoji = train
train_no_emoji['tweet'] = train_no_emoji['tweet'].apply(remove_emoji)

train.tail(10)

/var/folders/vb/9rxy0nsn1sz0zd7f5_x0rkyr0000gp/T/ipykernel_85766/3274737952.py:6: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(u'', tweet)


,id,tweet,subtask_a
7829,7991,@USER الله يسلمك يا منه يا رب امين يا رب,NOT
7830,7992,@USER يا عبدالعزيز يا حبك للصياح .. لك اكثر م...,NOT
7831,7993,إليك يا وجعي .. يا وجع الذكريات! URL,NOT
7832,7994,حبيب الروح يا حلو ابتسامك و يا محلى الليالي في...,NOT
7833,7995,يامن اسـمه دواء ، وذكره شفاء ، وطاعتـه غنى ، إ...,NOT
7834,7996,RT @USER: انتو بتوزعوا زيت وسكر فعلا يا عباس؟ ...,NOT
7835,7997,RT @USER: كدا يا عمر متزعلهاش يا حبيبي URL,NOT
7836,7998,هدا سكن اطفال امارتين من شارقة طالبين فزعتكم ي...,NOT
7837,7999,RT @USER: ومدني بمدد من قوتك أواجه به ضعفي.. و...,NOT
7838,8000,يا سلاام يا يو خالد انت والطرب الاصيل URL,NOT


# Sanity check

In [85]:
def count_chars(tweet):
    return len(tweet)

train['char_count'] = train['tweet'].apply(count_chars)
train_no_emoji['char_count'] = train_no_emoji['tweet'].apply(count_chars)

train.sort_values(by='char_count', ascending=[0]).head(10)

,id,tweet,subtask_a,char_count
2922,2946,من المظاهر المؤسفه ان الاب والام يقعدون يتها...,NOT,5623
6062,6160,جده الحُب ،، جده يا مُنتهى كل الكلام يا سيّدة...,NOT,5251
4761,4833,"يا مالي عمري رضا يا شبيه الورد , يا معنى الحي...",NOT,3000
6564,6707,يا من أظهر الجميل.. وستر القبيح.. يا من لا يؤا...,NOT,2202
2753,2760,يا مُكعب السُكرَ يا زمُردي الأحمر و يأسريُ ال...,NOT,2065
2124,2125,اللهم أني أسألك مسألة البائس الفقير وأدعوك دع...,OFF,1072
1399,1400,يا مرحبا ترحيبة عقاب يا نوت والحي يلحق لو تحده...,NOT,284
3050,3122,أنت وامثالك وحكامك وعلمائك الزنادقه خنازير دمر...,OFF,284
2434,2441,يا سيادة الرئيس يا سيادة الرئيس نعلم ان الحرب ...,OFF,284
6364,6507,بذلة عسكرية وسط الزحام .. حياة مليئة بالأشواك ...,NOT,283


Something is weird with these 6 tweets that have 1000+ characters. Those with 281-284 might be explained by my replacing '\<LF\>' with a space to keep word boundaries intact, but 1000+ is clearly some kind of data reading error. I'll just remove them.

In [86]:
train.drop(train[train['char_count'] > 1000].index, inplace=True)
train_no_emoji.drop(train_no_emoji[train_no_emoji['char_count'] > 1000].index, inplace=True)

train.reset_index(drop=True, inplace=True)
train_no_emoji.reset_index(drop=True, inplace=True)

train.sort_values(by='char_count', ascending=[0]).head(10)

,id,tweet,subtask_a,char_count
3050,3122,أنت وامثالك وحكامك وعلمائك الزنادقه خنازير دمر...,OFF,284
2434,2441,يا سيادة الرئيس يا سيادة الرئيس نعلم ان الحرب ...,OFF,284
1399,1400,يا مرحبا ترحيبة عقاب يا نوت والحي يلحق لو تحده...,NOT,284
6364,6507,بذلة عسكرية وسط الزحام .. حياة مليئة بالأشواك ...,NOT,283
7196,7358,يا عيباه يا حسافاه اليمني بيهان بكل مكان وهذا ...,NOT,283
1880,1881,يا راجل يا حمار هو عمك السيسي محتاج عياله يعمل...,OFF,283
475,476,الرساله فيها عشم مش طبيعي بس مش مشكلتي. اولا ك...,OFF,282
7740,7902,#قروب_المشتاقون_للجنه #عبدالرحمن_الخشتي اللهم ...,NOT,282
2654,2661,الهلال أمانة في يديكم حنا عند الله ثم عندكم قب...,NOT,282
7618,7780,القيناوية غاليين عليا اشرفها مية مية يا خلق يا...,NOT,282


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.countplot(train.subtask_a)
plt.title('count NOT/OFF')  # Much more NOT than OFF tweets

In [11]:
# Define Arabic and English punctuation to remove
import string
import re
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    """
    text is an arabic string input

    the preprocessed text is returned
    """
    
    # remove punctuation
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    # remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split())

    return text
  
train['tweet'] = train['tweet'].apply(preprocess)
print(train.head(5))

        id                                              tweet subtask_a  \
2922  2946  LFمن المظاهر المءسفه ان الاب والام LFيقعدون يت...       NOT   
6062  6160  LFجده الحب LFجده يا منتهي كل الكلام يا سيده كل...       NOT   
4761  4833  يا مالي عمري رضا يا شبيه الورد يا معني الحياه♥...       NOT   
6564  6707  يا من اظهر الجميل وستر القبيح يا من لا يءاخذ ب...       NOT   
2753  2760  يا مكعب السكر يا زمردي الاحمر و ياسري الاكبر ي...       NOT   

      word_count  char_count  avg_char_per_word  emoji_count  
2922         941        5857           4.403137           74  
6062         862        5416           4.437751           36  
4761         531        3178           4.226974          108  
6564         367        2283           4.397163            8  
2753         363        2129           4.165049           34  


In [41]:
# removing emojis 
# def remove_emoji(tweet):
#     return emoji.get_emoji_regexp().sub(u'', tweet)
    # return emoji.replace_emoji(tweet, replace='')

# removing USER
def user_remove(tweet):
  tweet = re.sub('(USER)', '', tweet)
  return tweet

train['tweet'] = train['tweet'].apply(remove_emoji)
train['tweet'] = train['tweet'].apply(user_remove)

train.head(40)

/var/folders/vb/9rxy0nsn1sz0zd7f5_x0rkyr0000gp/T/ipykernel_85766/71229768.py:15: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(u'', tweet)


,id,tweet,subtask_a
0,1,الحمدلله يارب فوز مهم يا زمالك.. كل الدعم ليكم...,NOT
1,2,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه,NOT
2,3,RT @USER: يا رب يا واحد يا أحد بحق يوم الاحد ا...,OFF
3,4,RT @USER: #هوا_الحرية يا وجع قلبي عليكي يا امي...,NOT
4,5,يا بكون بحياتك الأهم يا إما ما بدي أكون,NOT
5,6,@USER اخخ يا قلببي يا هالحلقه متعه على بكاء ع...,NOT
6,7,والله الزول السوداني أثبت أنه سابق بالتحضّر عن...,NOT
7,8,"RT @USER: جالس أسمع أحمد قاسم يغني: ""أحبك من ك...",NOT
8,9,في قلبي يا مغلاك وبعيني يا محلاك ..,NOT
9,10,يا التاج ع الراس يا السادة يا مالك الروح وراعيها,NOT


In [ ]:
from sklearn.model_selection import train_test_split
data = train
X = data.tweet.values
y = data.subtask_a.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2020)


In [32]:
# removing emojis 
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

# removing USER
def user_remove(string):
  string = re.sub('(USER)', '', string)
  return string
train['tweet'] = train['tweet'].apply(remove_emoji)
train['tweet'] = train['tweet'].apply(user_remove)
train.head(40)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  This is separate from the ipykernel package so we can avoid doing imports until


,id,tweet,subtask_a,word_count,char_count,avg_char_per_word,stopwords,emoji_count
2922,2946,LFمن المظاهر المءسفه ان الاب والام LFيقعدون يت...,NOT,941,5857,4.403137,211,74
6062,6160,LFجده الحب LFجده منتهي الكلام سيده المدن URL ...,NOT,862,5416,4.437751,203,36
4761,4833,مالي عمري رضا شبيه الورد معني الحياه NOT 4834 ...,NOT,531,3178,4.226974,147,108
6564,6707,اظهر الجميل وستر القبيح يءاخذ بالجريره يهتك ال...,NOT,367,2283,4.397163,102,8
2753,2760,مكعب السكر زمردي الاحمر ياسري الاكبر فتنتي الع...,NOT,363,2129,4.165049,79,34
2124,2125,اللهم اني اسالك مساله الباءس الفقير وادعوك دعا...,OFF,179,1113,4.673469,36,3
5316,5414,رنا رنا رنا رنا رنا رنا رنا رنا رنا رنا رنا رن...,NOT,84,279,2.333333,56,0
6428,6571,باي باي باي باي باي باي باي باي باي باي باي با...,NOT,78,272,2.500000,39,0
4030,4102,فهد باص فهد باص فهد باص فهد باص فهد باص فهد با...,OFF,74,276,2.743243,24,0
3777,3849,رب اتسفه رب اتسفه رب اتسفه رب اتسفه رب اتسفه ر...,NOT,73,278,2.821918,36,0


In [ ]:

from sklearn.model_selection import train_test_split
data = train
X = data.tweet.values
y = data.subtask_a.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2020)
